Bước 1 - Có thể không cần - Thực hiện Mount dữ liệu từ drive

In [ ]:
# Connect Driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Bước 2 - Import thư viện

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


Bước 3 - Khai báo các features

In [ ]:
CSV_HEADER=['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'Label', 'attack_cat'] # bo Src_IP, Dst_IP, Timestamp, {'Flow ID', 'Src Port', 'Dst Port', 'Protocol'}
CSV_HEADER_WITHOUT_LABEL = ['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

# Bảng mã hóa
encoded_to_original = {
    0: 'Benign',
    1: 'Analysis',
    2: 'Backdoor',
    3: 'DoS',
    4: 'Exploits',
    5: 'Fuzzers',
    6: 'Generic',
    7: 'Reconnaissance',
    8: 'Shellcode',
    9: 'Worms'
}
# decoded_labels = [encoded_to_original[label] for label in y_pred]
# decoded_labels

In [ ]:
!pip install flwr


bước 5 - Đọc chia dữ liệu

In [ ]:
#df = pd.read_csv("/content/drive/MyDrive/ATHTTT/CIC_UNSW-LABEL_Attack_cat.csv")
df = pd.read_csv('/content/drive/MyDrive/AnToanHeThongThongTin-CH2024/CIC_UNSW-LABEL_Attack_cat.csv')
df = df.drop(['attack_cat'], axis=1)

scaler = StandardScaler()
scaler.fit(df[CSV_HEADER_WITHOUT_LABEL])
df[CSV_HEADER_WITHOUT_LABEL] = scaler.transform(df[CSV_HEADER_WITHOUT_LABEL])

# Chia dữ liệu thành train/test
train_data, test_data = train_test_split(df, test_size=0.2)
train_data_1, train_data_2 = train_test_split(train_data, test_size=0.5)

# Lưu dữ liệu vào các tệp
train_data_1.to_csv("train_data_1.csv", index=False, header=True)
train_data_2.to_csv("train_data_2.csv", index=False, header=True)
train_data.to_csv("train_data.csv", index=False, header=True)
test_data.to_csv("test_data.csv", index=False, header=True)

# Xác định đặc trưng (X) và nhãn (y)
X_train_1 = train_data_1[CSV_HEADER_WITHOUT_LABEL]
y_train_1 = train_data_1['Label']

X_train_2 = train_data_2[CSV_HEADER_WITHOUT_LABEL]
y_train_2 = train_data_2['Label']

X_test = test_data[CSV_HEADER_WITHOUT_LABEL]
y_test = test_data['Label']

Bước 6 - Thực hiện train dữ liệu - mô phỏng Federated Learning với thuật toán FEDERATED AVERAGING

In [ ]:

# Huấn luyện mô hình Logistic Regression cho client 1 và client 2
model_1 = LogisticRegression(max_iter=1000)
model_1.fit(X_train_1, y_train_1)

model_2 = LogisticRegression(max_iter=1000)
model_2.fit(X_train_2, y_train_2)

# Lấy các tham số từ mô hình của mỗi client
coef_1 = model_1.coef_
intercept_1 = model_1.intercept_

coef_2 = model_2.coef_
intercept_2 = model_2.intercept_

# Tổng hợp trọng số theo công thức Federated Averaging (FedAvg) mà không tính trọng số
avg_coef = (coef_1 + coef_2) / 2
avg_intercept = (intercept_1 + intercept_2) / 2

# Tạo mô hình mới với trọng số tổng hợp
model_avg = LogisticRegression()
model_avg.coef_ = avg_coef
model_avg.intercept_ = avg_intercept
model_avg.classes_ = model_1.classes_  # hoặc model_2.classes_, vì chúng phải giống nhau

# Dự đoán và đánh giá mô hình trên tập test
y_pred = model_avg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Federated Averaging model: {accuracy}")

# Đánh giá hiệu suất mô hình
precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
recall = recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
f1 = f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

# Tính Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# In báo cáo phân loại (classification report)
report = classification_report(y_test, y_pred, target_names=encoded_to_original.values())
print("\nClassification Report:\n", report)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Accuracy of Federated Averaging model: 0.8970898496366498
Accuracy: 89.71%
Precision: 0.92
Recall: 0.90
F1-score: 0.90

Confusion Matrix:
[[69558     0     0     0   213  1547     7   139     0     0]
 [    2     0     0     0    90     0     0     0     0     0]
 [    1     0     4     0    30    55     2     4     0     0]
 [    7     0     3    19   487   306     5    59     0     0]
 [   71     0     0    10  3804  2140    59    89     0     0]
 [   20     0     0     2   449  4845    21   718     0     0]
 [   20     0     0     5   371   353   202    28     0     0]
 [   84     0     0     0    41  1267     0  1932     0     0]
 [    3     0     0     0    18   313     0   119     0     0]
 [    0     0     0     0    23    32     0     6     0     0]]

Classification Report:
                 precision    recall  f1-score   support

        Benign       1.00      0.97      0.99     71464
      Analysis       0.00      0.00      0.00        92
      Backdoor       0.57      0.04  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
